In [50]:
import numpy as np
import scipy.io as spio
import os.path as path
from os import listdir
from os.path import isfile, join
import os
import timeit as tt
import json
import sys
#import pandas
from tqdm import tqdm
import csv
from datetime import date
from datetime import datetime
np.set_printoptions(threshold=sys.maxsize)

In [51]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [52]:
def sigmoid(X):
    return 1/(1+np.exp(-X))

def relu(X):
    return np.maximum(0,X)

def softmax(X):
    expo = np.exp(X)
    expo_sum = np.sum(np.exp(X))
    return expo/expo_sum

In [53]:
def getFilesInFloder(folderPath):
    onlyfiles = [f for f in listdir(folderPath) if isfile(join(folderPath, f))]
    return onlyfiles

In [54]:
def genICFilterMatrix(bandnum, y_length, mid_freq_matrix):
    ret_icfilter = np.zeros((bandnum, y_length),dtype=float)
    for i in range(bandnum):
        if i == 9:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-360*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
        elif i == 10:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-360*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
        else:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-20*12*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
    
    return ret_icfilter

In [55]:
def scale_back_weight(srcWeight,bits):
    restored_weight = srcWeight/(2**bits)
    return restored_weight

In [63]:
def runAccuracyTestWithQuantWeights(weights_file=None, test_matfile=None, comp_base=1, threshold=-145, lbl_len=513,
                    icfilterary=None ,startPoint=1600, stepQuantity=400, record_csv=None \
                   ):
    """doing the accuracy of input matfile
    
    parameters:on-default argument fo
    test_matfile: to test mat file path
    camp_base: accuracy co
    quant_scale for w1 is
    5

    quant_scale for w2 is
    6

    quant_scale for w3 is
    5

    quant_scale for b1 is
    6

    quant_scale for b2 is
    7

    quant_scale for b3 is
    8
    """
    weights = spio.loadmat(weights_file)
    wscale_bits1 = 5
    wscale_bits2 = 6
    wscale_bits3 = 5
    bscale_bits1 = 6
    bscale_bits2 = 7
    bscale_bits3 = 8
    
    w1 = scale_back_weight(weights["w1"],wscale_bits1)
    w2 = scale_back_weight(weights["w2"],wscale_bits2)
    w3 = scale_back_weight(weights["w3"],wscale_bits3)
    b1 = scale_back_weight(weights["b1"],bscale_bits1)
    b2 = scale_back_weight(weights["b2"],bscale_bits2)
    b3 = scale_back_weight(weights["b3"],bscale_bits3)
    mat_data = spio.loadmat(test_matfile)
    
    inband = 4
    bandnum = 10
    count_answer_1 = 0
    count_answer_0 = 0
    count_test_1 = 0
    count_test_0 = 0

    y_answer = np.empty([0,0],dtype=int)
    y_test = np.empty([0,0])

    local_power = np.empty([0,0])
    local_power_normalize = np.empty([0,0],dtype=int)
    x_all = np.empty([0,0])
    x_normalize_all = np.empty([0,0])
    
    s_len = int(400*(np.floor(len(mat_data['s'])/400)))
    sample_num = inband * bandnum
    test_counter = 0
    csv_columns = ['test_no','prediction','answer']
    csv_raw_list = []
    for i in range(startPoint,s_len,stepQuantity):
        test_counter += 1
        record_dict = {}
        record_dict['test_no'] = test_counter
        #divide four segment every 25ms
        idx1 = i-1600
        idx2 = i-1200
        idx3 = i-800
        idx4 = i-400
        s1 = mat_data['s'][idx1:idx2] #0-399
        s2 = mat_data['s'][idx2:idx3] #400-799run_test_main,number
        s3 = mat_data['s'][idx3:idx4] #800-1199
        s4 = mat_data['s'][idx4:i]    #1200-1599
        loop_record={
            "s1":"{}_{}".format(idx1,idx2),
            "s2":"{}_{}".format(idx2,idx3),
            "s3":"{}_{}".format(idx3,idx4),
            "s4":"{}_{}".format(idx4,i)
        }
        loop_record["threshold"]=threshold      

        s_fft_1 = np.fft.fft(s1,n=1024,axis=0) 
        s_fft_2 = np.fft.fft(s2,n=1024,axis=0) 
        s_fft_3 = np.fft.fft(s3,n=1024,axis=0) 
        s_fft_4 = np.fft.fft(s4,n=1024,axis=0) 

        s_fft_1_513 = np.split(s_fft_1,[0,lbl_len-1],axis=0)[1]
        s_fft_2_513 = np.split(s_fft_2,[0,lbl_len-1],axis=0)[1]
        s_fft_3_513 = np.split(s_fft_3,[0,lbl_len-1],axis=0)[1]
        s_fft_4_513 = np.split(s_fft_4,[0,lbl_len-1],axis=0)[1]

        # there is one error element in every array in position 9 of each array
        x1 = np.log(np.matmul(icfilterary,(np.abs(s_fft_1_513)**2)) + 0.0001)
        x2 = np.log(np.matmul(icfilterary,(np.abs(s_fft_2_513)**2)) + 0.0001)
        x3 = np.log(np.matmul(icfilterary,(np.abs(s_fft_3_513)**2)) + 0.0001)
        x4 = np.log(np.matmul(icfilterary,(np.abs(s_fft_4_513)**2)) + 0.0001)

        x= np.array([x1,x2,x3,x4]).reshape(1,sample_num)
        x_40 = x[0:40]
        
        max_ele = np.amax(x_40,axis=1)
        min_ele = np.amin(x_40,axis=1)
        sum_of_x = np.sum(x_40)
        
        loop_record["max_ele"]=max_ele[0]
        loop_record["min_ele"]=min_ele[0]
        loop_record["sum_of_x"]=sum_of_x

        # normalize
        x_normalize = (x_40-min_ele)/(max_ele-min_ele+0.0001)

        #counting the test
        loop_record["answer"]=0
        if sum_of_x > threshold:
            #performing the model weight mulplications
            answer = softmax(np.matmul(relu(np.matmul(relu(np.matmul(x_normalize,w1)+b1),w2)+b2),w3)+b3)
            loop_record["answer"]=answer
            if answer[0,0] > answer[0,1]:
                y_test = np.append(y_test, 0)
                count_test_0 += 1
                record_dict['prediction'] = 0
            else:
                y_test = np.append(y_test, 1)
                count_test_1 += 1
                record_dict['prediction'] = 1
        else:
            loop_record["answer"]=9999
            y_test = np.append(y_test, 0)
            count_test_0 += 1
            record_dict['prediction'] = 0

        #counting our answer
        if comp_base == 1:
            if sum_of_x > threshold:
                y_answer = np.append(y_answer,1)
                count_answer_1 += 1
                record_dict['answer'] = 1
            else:
                y_answer = np.append(y_answer,0)
                count_answer_0 += 1
                record_dict['answer'] = 0
        else:
            y_answer = np.append(y_answer,0)
            count_answer_0 += 1
            record_dict['answer'] = 0
        csv_raw_list.append(record_dict)
        
    acc = 0
    LED = 0
    wrong = np.empty([0,0])
    y_answer_len = len(y_answer)    
    y_test_len = len(y_test)
    
    if comp_base == 1: #if our test data is human voice
        for c in range(y_answer_len):
            acc += abs(y_answer[c]-y_test[c])

        acc = (y_answer_len-acc)/y_answer_len
    else:
        acc = count_test_0/(count_test_0+count_test_1)
    
    with open(record_csv,"w") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in csv_raw_list:
            writer.writerow(data)
            
    print("\n{} test completed\n".format(test_matfile))
    print("acc is {}\n".format(acc))
    print("csv file output completed.\n location:{}\n".format(record_csv))  
    
    return acc #json.dumps(loop_record,cls=NumpyEncoder)


In [71]:
middleFreq = [16, 20, 26, 36, 48, 60, 80, 101, 256, 353]
_weights = "../../Weights/int8_weights_for_original_128x32_v3.mat"

y_len = 513
icfilterary = genICFilterMatrix(10, y_len, middleFreq)[:,1:]
def run_multitest_main():
    #get date and time string:
    dateStr = "{}".format(str(date.today()).replace("-", ""))
    timeStr = "{}".format(str(datetime.now().strftime("%H_%M_%S")))
    #create csv report saving directory
    upper_dir = "../Test_Reports/Test_int8_{}_{}/".format(dateStr,timeStr)
    if not path.exists(report_dir):
        os.makedirs(report_dir+"csv/speech/")
        os.makedirs(report_dir+"csv/noise/")
    speech_report_dir = upper_dir + "csv/speech/"
    noise_report_dir = upper_dir + "csv/noise/"
    
    #Testing Speech
    speech_matfilespath = "../../speechData/forKriz/clean_mat/"
    cflag = 1
    speech_filelist = getFilesInFloder(speech_matfilespath)
    print("Speech Files to testing are {}".format(speech_filelist))
    print("Starting Speech Test")
    all_speech_acc = []
    for sf in tqdm(speech_filelist):
        speech_csv_file_name = speech_report_dir+"int8_test_{}_CleanSpeech_{}_{}.csv".format(sf, dateStr,timeStr)
        s_acc = runAccuracyTestWithQuantWeights(_weights, speech_matfilespath+f, icfilterary=icfilterary, 
                                                  comp_base=cflag, record_csv=speech_csv_file_name)
        all_speech_acc.append(s_acc)
    
    #Testing Noise
    noise_matfilespath = "../../speechData/forKriz/noise_mat/"
    cflag = 0
    noise_filelist = getFilesInFloder(noise_matfilespath)
    print("Noise Files to testing are {}".format(noise_filelist))
    print("starting Noise Test")
    all_noise_acc = []
    for nf in tqdm(noise_filelist):
        noise_csv_file_name = noise_report_dir+"int8_test_{}_Noise_{}_{}.csv".format(sf, dateStr,timeStr)
        n_acc = runAccuracyTestWithQuantWeights(_weights, speech_matfilespath+f, icfilterary=icfilterary, 
                                                  comp_base=cflag, record_csv=noise_csv_file_name)
        all_speech_acc.append(s_acc)
    
                                                                           
    for f in tqdm(filelist):
        csv_file_name = None
        if cflag == 1:
            csv_file_name = report_dir+"int8_test_{}_CleanSpeech_{}_{}.csv".format(f, dateStr,timeStr)
        else:
            csv_file_name = report_dir+"int8_test_{}_int8_Noise_{}_{}.csv".format(f, dateStr,timeStr)
            
        _acc = runAccuracyTestWithQuantWeights(_weights, _matfilespath+f, icfilterary=icfilterary, 
                                                  comp_base=cflag, record_csv=csv_file_name)
        all_acc.append(_acc)
    
    sum_of_acc = sum(all_acc)
    avg_acc = sum_of_acc/flistlen
    print("\nThe average accuarncy(int8 weight) of testing {} speech files is {}\n".format(flistlen,avg_acc))
        

/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in log10
  if sys.path[0] == '':
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log10
  


In [72]:
run_multitest_main()

 12%|█▏        | 2/17 [00:00<00:01, 12.65it/s]

files in _matfilespath are ['銅摩擦聲.mp3.wav.mat', '窗簾2.mp3.wav.mat', '拖拉辦公椅.mp3.wav.mat', '關門聲.mp3.wav.mat', '電話按鍵.mp3.wav.mat', '拍身體1.mp3.wav.mat', '揉塑膠袋.mp3.wav.mat', '敲玻璃.mp3.wav.mat', '椅子聲.mp3.wav.mat', '拍手2.mp3.wav.mat', '麥克筆敲擊1.mp3.wav.mat', '插座敲擊.mp3.wav.mat', '電燈開關.mp3.wav.mat', '腳步聲.mp3.wav.mat', '磁鐵敲擊.mp3.wav.mat', '插座開關.mp3.wav.mat', '板擦拍敲桌子.mp3.wav.mat']
starting to test

../../speechData/forKriz/noise_mat/銅摩擦聲.mp3.wav.mat test completed

acc is 0.7777777777777778

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_銅摩擦聲.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/窗簾2.mp3.wav.mat test completed

acc is 0.9444444444444444

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_窗簾2.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv



 29%|██▉       | 5/17 [00:00<00:00, 14.29it/s]


../../speechData/forKriz/noise_mat/拖拉辦公椅.mp3.wav.mat test completed

acc is 0.7904040404040404

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_拖拉辦公椅.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/關門聲.mp3.wav.mat test completed

acc is 0.8493150684931506

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_關門聲.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/電話按鍵.mp3.wav.mat test completed

acc is 0.8838383838383839

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_電話按鍵.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/拍身體1.mp3.wav.mat test completed

acc is 0.9090909090909091

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_拍身體1.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv



 53%|█████▎    | 9/17 [00:00<00:00, 14.62it/s]


../../speechData/forKriz/noise_mat/揉塑膠袋.mp3.wav.mat test completed

acc is 0.5277777777777778

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_揉塑膠袋.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/敲玻璃.mp3.wav.mat test completed

acc is 0.5808080808080808

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_敲玻璃.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/椅子聲.mp3.wav.mat test completed

acc is 0.8686868686868687

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_椅子聲.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/拍手2.mp3.wav.mat test completed

acc is 0.8914141414141414

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_拍手2.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv



 76%|███████▋  | 13/17 [00:00<00:00, 14.13it/s]


../../speechData/forKriz/noise_mat/麥克筆敲擊1.mp3.wav.mat test completed

acc is 0.9166666666666666

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_麥克筆敲擊1.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/插座敲擊.mp3.wav.mat test completed

acc is 0.8914141414141414

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_插座敲擊.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/電燈開關.mp3.wav.mat test completed

acc is 0.8106060606060606

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_電燈開關.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv



100%|██████████| 17/17 [00:01<00:00, 15.05it/s]


../../speechData/forKriz/noise_mat/腳步聲.mp3.wav.mat test completed

acc is 0.8282828282828283

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_腳步聲.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/磁鐵敲擊.mp3.wav.mat test completed

acc is 0.8863636363636364

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_磁鐵敲擊.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/插座開關.mp3.wav.mat test completed

acc is 0.8005050505050505

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_插座開關.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


../../speechData/forKriz/noise_mat/板擦拍敲桌子.mp3.wav.mat test completed

acc is 0.7904040404040404

csv file output completed.
 location:../Test_Reports/Test_Noise_20200319_16_03_45/csv/int8_test_板擦拍敲桌子.mp3.wav.mat_int8_Noise_20200319_16_03_45.csv


The average accuarncy(int